# Parameters generator

The SM will be defined as pandas file in JSON format.
In this file it will be shown the definition of such a file from a dictionary

In [2]:
import pandas as pd

TODO: Capture also main configuration files in parent directory

SMMain={'ParticleDefinitionsMain':[{'Description':{'Neutral Higgs':{'PDG':[0],'Width':0}}}]}
SMMain['ParticleDefinitionsMain'][0]['Description']['Neutral Higgs']

Build SM as dictionary to feed pandas

In [559]:
#Matter fields
# LaTeX backslash as \\\\
str_keys=['LaTeX','OutputName','Description']
lst_keys=['PDG','PDG.IX']
ParticleDefinitions={'GaugeES':
                             {'H0':{'PDG'  :[0],
                                    'Width':0,
                                    'Mass' : 'Automatic',
                                    'FeynArtsNr' : 1,
                                    'LaTeX' : 'H^0',
                                    'OutputName' : 'H0'                                   
                                   },
                              'Hp':{'PDG'  :[0],
                                    'Width':0,
                                    'Mass' : 'Automatic',
                                    'FeynArtsNr' : 2,
                                    'LaTeX' : 'H^+',
                                    'OutputName' : 'Hp'                                   
                                   },
                              'VB' : {'Description' :'B-Boson'      },                                                   
                              'VG' : {'Description' :'Gluon'        },          
                              'VWB': {'Description' :'W-Bosons'     },          
                              'gB' : {'Description' :'B-Boson Ghost'},                                                   
                              'gG' : {'Description' :'Gluon Ghost'  },          
                              'gWB': {'Description' :'W-Boson Ghost'}
                             },
                            'EWSB':
                            {                                
                            },
                            'WeylFermionAndIndermediate':
                            {   
                            }
                           }

ParticleDefinitions['EWSB']['hh']={'Description' : 'Higgs',
                                   'PDG'         : [25],
                                   'PDG.IX'      : [101000001]}
ParticleDefinitions['EWSB']['Ah']={'Description' : "Pseudo-Scalar Higgs",
                                   'PDG'         : [0],
                                   'PDG.IX'      : [0],
                                   'Mass'        : [0],
                                   'Width'       : [0]
                                   }
ParticleDefinitions['EWSB']['Hp']={'Description'    : 'Charged Higgs', 
                                   'PDG'            : [0],
                                   'PDG.IX'         : [0],
                                   'Width'          : [0], 
                                   'Mass'           : [0],
                                   'LaTeX'          : ['H^+','H^-' ],
                                   'OutputName'     : ['Hp' ,'Hm'  ],
                                   'ElectricCharge' : 1
                                   }

ParticleDefinitions['EWSB']['VP']  =  { 'Description' : 'Photon'}
ParticleDefinitions['EWSB']['VZ']  =  { 'Description' : 'Z-Boson',
                                        'Goldstone' : 'Ah' }
ParticleDefinitions['EWSB']['VG']  =  { 'Description' : 'Gluon' }
ParticleDefinitions['EWSB']['VWp'] =  { 'Description' : 'W+ - Boson',
                                        'Goldstone': 'Hp' }
ParticleDefinitions['EWSB']['gP']  =  { 'Description' : 'Photon Ghost'}
ParticleDefinitions['EWSB']['gWp'] =  { 'Description' : 'Positive W+ - Boson Ghost'}
ParticleDefinitions['EWSB']['gWpC']=  { 'Description' : 'Negative W+ - Boson Ghost' }
ParticleDefinitions['EWSB']['gZ']  =  { 'Description' : 'Z-Boson Ghost' }
ParticleDefinitions['EWSB']['gG']  =  { 'Description' : 'Gluon Ghost' }
ParticleDefinitions['EWSB']['Fd']  =  { 'Description' : 'Down-Quarks'}
ParticleDefinitions['EWSB']['Fu']  =  { 'Description' : 'Up-Quarks'}
ParticleDefinitions['EWSB']['Fe']  =  { 'Description' : 'Leptons' }
ParticleDefinitions['EWSB']['Fv']  =  { 'Description' : 'Neutrinos' }

ParticleDefinitions['WeylFermionAndIndermediate']['H' ] = {'PDG'       : [0],
                                                           'Width'     : 0,
                                                           'Mass'      : 'Automatic',
                                                           'LaTeX'     : 'H',
                                                           'OutputName': '' }
ParticleDefinitions['WeylFermionAndIndermediate']['dR'] = {'LaTeX' : 'd_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['eR'] = {'LaTeX' : 'e_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['l' ] = {'LaTeX' : 'l'     }
ParticleDefinitions['WeylFermionAndIndermediate']['uR'] = {'LaTeX' : 'u_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['q' ] = {'LaTeX' : 'q'     }
ParticleDefinitions['WeylFermionAndIndermediate']['eL'] = {'LaTeX' : 'e_L'   }
ParticleDefinitions['WeylFermionAndIndermediate']['dL'] = {'LaTeX' : 'd_L'   }
ParticleDefinitions['WeylFermionAndIndermediate']['uL'] = {'LaTeX' : 'u_L'   }
ParticleDefinitions['WeylFermionAndIndermediate']['vL'] = {'LaTeX' : '\\\\nu_L'}
ParticleDefinitions['WeylFermionAndIndermediate']['DR'] = {'LaTeX' : 'D_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['ER'] = {'LaTeX' : 'E_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['UR'] = {'LaTeX' : 'U_R'   }
ParticleDefinitions['WeylFermionAndIndermediate']['EL'] = {'LaTeX' : 'E_L'   }
ParticleDefinitions['WeylFermionAndIndermediate']['DL'] = {'LaTeX' : 'D_L'   }
ParticleDefinitions['WeylFermionAndIndermediate']['UL'] = {'LaTeX' : 'U_L'   }

Example to modify dictionary entry

In [562]:
ParticleDefinitions['GaugeES']['H0']['FeynArtsNr']=1

## Save as JSON

In [563]:
f=open('SM.json','w')
json.dump(ParticleDefinitions,f)
f.close()

Check if `pandas` can read it

In [571]:
pd.read_json('kk.json').loc['Hp']['EWSB']

{'Description': 'Charged Higgs',
 'ElectricCharge': 1,
 'LaTeX': ['H^+', 'H^-'],
 'Mass': [0],
 'OutputName': ['Hp', 'Hm'],
 'PDG': [0],
 'PDG.IX': [0],
 'Width': [0]}

## Feed a `pandas` `DataFrame`

In [594]:
SM=pd.DataFrame( ParticleDefinitions  )

Checks

In [595]:
SM['GaugeES']['H0']

{'FeynArtsNr': 1,
 'LaTeX': 'H^0',
 'Mass': 'Automatic',
 'OutputName': 'H0',
 'PDG': [0],
 'Width': 0}

In [596]:
SM['EWSB']['VZ']

{'Description': 'Z-Boson', 'Goldstone': 'Ah'}

In [597]:
SM[:1]

,EWSB,GaugeES,WeylFermionAndIndermediate
Ah,"{'Mass': [0], 'PDG.IX': [0], 'PDG': [0], 'Desc...",NaN,NaN


Show a particle

In [598]:
SM['GaugeES']['H0']

{'FeynArtsNr': 1,
 'LaTeX': 'H^0',
 'Mass': 'Automatic',
 'OutputName': 'H0',
 'PDG': [0],
 'Width': 0}

Example: Modify particle property

In [599]:
SM['WeylFermionAndIndermediate']['dR'] = {'LaTeX' : 'd_R'   }

In [600]:
SM.to_json('SM2.json')#,lines=True,orient='records')

## Load JSON
Directly fromo the `json.dumps...`

In [602]:
BSM=pd.read_json('SM.json')#,lines=True,orient='records')

In [603]:
BSM[:1]

,EWSB,GaugeES,WeylFermionAndIndermediate
Ah,"{'Mass': [0], 'PDG.IX': [0], 'PDG': [0], 'Desc...",NaN,NaN


In [604]:
BSM['GaugeES'].dropna()['H0']

{'FeynArtsNr': 1,
 'LaTeX': 'H^0',
 'Mass': 'Automatic',
 'OutputName': 'H0',
 'PDG': [0],
 'Width': 0}

In [605]:
BSM['GaugeES']['H0']

{'FeynArtsNr': 1,
 'LaTeX': 'H^0',
 'Mass': 'Automatic',
 'OutputName': 'H0',
 'PDG': [0],
 'Width': 0}

In [606]:
BSM['EWSB']['VZ']

{'Description': 'Z-Boson', 'Goldstone': 'Ah'}

In [607]:
#SM.loc['Ah','EWSB']['Mass']=[1]

Still working...

In [608]:
BSM['WeylFermionAndIndermediate']['dR'] = {'LaTeX' : 'd_R'   }

In [609]:
BSM.loc['Ah','EWSB']

{'Description': 'Pseudo-Scalar Higgs',
 'Mass': [0],
 'PDG': [0],
 'PDG.IX': [0],
 'Width': [0]}

## Export to files

In [610]:
SM=BSM.loc[['Ah', 'DL', 'DR', 'EL', 'ER', 'Fd', 'Fe', 'Fu', 'Fv', 'H', 'H0', 'Hp',
       'UL', 'UR', 'VB', 'VG', 'VP', 'VWB', 'VWp', 'VZ', 'dL', 'dR', 'eL',
       'eR', 'gB', 'gG', 'gP', 'gWB', 'gWp', 'gWpC', 'gZ', 'hh', 'l', 'q',
       'uL', 'uR', 'vL']]

In [612]:
CHECK_PANDAS_SAVED_JSON=True
if CHECK_PANDAS_SAVED_JSON:
    SM=pd.read_json('SM2.json')#,lines=True,orient='records')

In [613]:
import json
f=open('p.m','w')
for c in SM.columns:
    f.write('ParticleDefinitions[{}] = {{\n'.format(c))
    cindex=SM[c].dropna().index
    csep=','
    for p in cindex:
        if p==list(cindex)[-1]:
            csep=''
        f.write('    {{{}, {{'.format(p))
        cpkeys=SM[c][p].keys()
        sangria=' '
        sep=','
        nl='\n'
        #TODO: Be sure that Description will be printed first
        for k in cpkeys:
            if k!=list(cpkeys)[0]:
                sangria='           '
            if k==list(cpkeys)[-1]:
                sep=''
                nl=''
            #print list of strings with double quotes
            cp='{}{} -> '.format(sangria,k)
            smcpk=SM[c][p][k]
            if type(smcpk)==str:
                smcpk='"{}"'.format(smcpk)
            else:
                smcpk=json.dumps( smcpk )
            
            f.write('{}{}{}{}'.format(cp,smcpk,sep,nl))

        f.write('}}}}{}\n'.format(csep))
    f.write('};\n\n')
f.close()

In [614]:
cat p.m

ParticleDefinitions[EWSB] = {
    {Ah, { Mass -> [0],
           PDG.IX -> [0],
           PDG -> [0],
           Description -> "Pseudo-Scalar Higgs",
           Width -> [0]}},
    {Fd, { Description -> "Down-Quarks"}},
    {Fe, { Description -> "Leptons"}},
    {Fu, { Description -> "Up-Quarks"}},
    {Fv, { Description -> "Neutrinos"}},
    {Hp, { Width -> [0],
           Mass -> [0],
           LaTeX -> ["H^+", "H^-"],
           PDG.IX -> [0],
           PDG -> [0],
           ElectricCharge -> 1,
           OutputName -> ["Hp", "Hm"],
           Description -> "Charged Higgs"}},
    {VG, { Description -> "Gluon"}},
    {VP, { Description -> "Photon"}},
    {VWp, { Goldstone -> "Hp",
           Description -> "W+ - Boson"}},
    {VZ, { Goldstone -> "Ah",
           Description -> "Z-Boson"}},
    {gG, { Description -> "Gluon Ghost"}},
    {gP, { Description -> "Photon Ghost"}},
    {gWp, { Description -> "Positive W+ - Boson Ghost"}},
    {gWpC, { Description -> "Negative W+ - Bo

# Get particle from model file

Design:
* Intialize `particle` as an empty dictionary.
* Intialize `particles` as an empty list.
* Fill particle properties and build the list: `particles`
* Internal defintions
  * weyl: Charged Weyl fermion bidiagonal rotation object
  * symm: Symmetric diagonalization system object
  * vev:  Complex scalar decomposition

Implementation:
1. Get particles from particles definitions
2. Get particles from `DEFINITION[EWSB]`

## Load Model file

In [64]:
pwd

'/home/restrepo/tmp'

In [65]:
MODEL_PATH="./SARAH/Models/SM/SM.m"

In [66]:
f=open(MODEL_PATH,'r')

In [67]:
smf=f.read()
f.close()

In [68]:
smf

'Off[General::spell]\n\nModel`Name = "SM";\nModel`NameLaTeX ="Standard Model";\nModel`Authors = "F.Staub";\nModel`Date = "2014-11-06";\n\n(* 2013-01-24: changed normalization of lambda term to convention of hep-ph/0207271 *)\n(* 2013-06-24: using new name conventions (without inital "S" and "F" for scalar and matter fields) *)\n(* 2013-09-01: changing to new conventions for FermionFields/MatterFields *)\n(* 2014-11-06: Changed sign in Lagrangian to fit standard conventions *)\n(* 2016-05-03: Changed sign of Yu *)\n\n\n\n(*-------------------------------------------*)\n(*   Particle Content*)\n(*-------------------------------------------*)\n\n(* Gauge Groups *)\n\nGauge[[1]]={B,   U[1], hypercharge, g1,False};\nGauge[[2]]={WB, SU[2], left,        g2,True};\nGauge[[3]]={G,  SU[3], color,       g3,False};\n\n\n(* Matter Fields *)\n\nFermionFields[[1]] = {q, 3, {uL, dL},     1/6, 2,  3};  \nFermionFields[[2]] = {l, 3, {vL, eL},    -1/2, 2,  1};\nFermionFields[[3]] = {d, 3, conj[dR],     1

## General definition

In [69]:
import re
import cmdlike as cmd
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_colwidth',200)

In [70]:
#TODO: Initialize objects
particles=[]
particlessons=[]

Fields=['Gauge','FermionFields','ScalarFields']
NAME='DEFINITION'
KEY='EWSB'
# Get also particles from
#NAME[KEY] dictionary!
def get_particles(fdotm,Fields,NAME,KEY,particles,particlessons):
    '''
    Extract particles from  SARAH Model files by using the information
    from: 
    * Fields: list of lists
    * NAME[KEY] dictionary 
    '''
    #i=1
    #Field=Fields[i]
    for Field in Fields:
        for f in cmd.grep(Field,fdotm).split('\n'):
            particle={}
            if not re.search('^\s*\(\*',f):
                #Fix components fields
                ff=f.split('{')
                if len(ff) ==3:
                    fff=ff[2].split('}')
                    f=ff[0]+'{'+ff[1]+fff[0].replace(',','::')+''.join(fff[1:])+'};'
                g=re.search('%s\[\[[0-9]+\]\]\s*=\s*\{(.*)\s*\}\s*;\s*' 
                    %Field,f)

                if g:
                    try:
                        fp=g.groups()[0].split(',')
                    except 'KeyError':
                        fp=[]
                    if len(fp)>0:
                        print(g.groups(),Field,len(fp))
                        particle['Field']=fp[0].strip()
                        particle['Parents']=None
                        particle['Properties']={}
                    if Field=='Gauge' and len(fp)>=5:
                        particle['Field']='V'+particle['Field']
                        particle['Properties']['Group']=fp[1]
                        particle['Definition']='GaugeES'
                        particle['Properties']['Index']=fp[2]
                        particle['Properties']['Coupling']=fp[3]
                        particle['Properties']['SSB']=fp[3]
                        particle['Properties']['Lorentz']='Vector'
                        print(particle)
                    else: 
                        if len(fp)>=6:
                            particle['Properties']['NF']=fp[1].strip()
                            particle['Properties']['Groups']=fp[3:]
                            particle['Definition']='WeylFermionAndIndermediate'
                            if Field=='FermionFields':
                                particle['Properties']['Lorentz']='WeylFermion'
                            elif Field=='ScalarFields':
                                particle['Properties']['Lorentz']='Scalar'
                            print("**********")
                            sons=re.sub('conj\[(\w+)\]',r'\1', fp[2] ).split('::')
                            print(fp[2],sons)
                            for s in sons:
                                print(s)
                                particleson={}
                                particleson['Properties']={}
                                particleson['Field']=s.strip()
                                if Field=='FermionFields':
                                    particleson['Definition']='WeylFermionAndIndermediate'
                                    particleson['Properties']['Lorentz']='WeylFermion'
                                elif Field=='ScalarFields':
                                    particleson['Definition']='GaugeES'
                                    particleson['Properties']['Lorentz']='Scalar'
                                particleson['Parents']=particle['Field']
                                particleson['Properties']['NF']=particle['Properties']['NF']
                                particlessons=particlessons+[particleson]
                            print("**********")

                    particles.append(particle)
    particles=particles+particlessons
    return particles

def sarahlist_to_python(strl,only_extract=False,DEBUG=False):
    '''
    Convert a string with a SARAH list of rotations into
    a Python object
    '''
    if only_extract:
        return strl
    #General transformations
    nl=re.sub(';\s*$','', #Drop final semicolon
               strl)
    if ( re.search('^\s*\t*\{.*\{.*\{.*->True',strl ) or 
       re.search('^\s*\t*\{.*\{.*\{.*->False',strl ) ):
        nl=re.sub( '([\w]+)([,:])',r'"\1"\2', # Keep True and False
              re.sub( '[\s\t]+','', # strip
              re.sub( '\s*->\s*',':', # to python dict                  
                  nl))).replace('}},{','}],['
                      ).replace('{{','[['
                      ).replace('}}}','}]]'
                      )        
    elif nl.find('->')>-1:
        nl=re.sub( '(\w+\[*\w+\]*)',r'"\1"' ,
           re.sub('\s*:\s*\{([\s\w,\[\]]+)\}',r':[\1 ]', # to python value lists of dict key
           re.sub( '\s*->\s*',':', # to python dict                  
              nl)))
    else:
        nl= re.sub( '\{','[',
            re.sub( '\}',']',                   
            re.sub('([\w\[\]\/\\\]+)',r'"\1"',  #to python_lists
           nl)))
    if DEBUG:    
        print(nl)
        print("*"*50)
    return eval(nl)

def extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract=False):
    dsbd={}
    start=False
    startlist=False
    fields=''
    dsbt=''
    for f in fdotm.split('\n'):
        dsb=re.search(pattern,f)
        if dsb:
            #Capture EWSB DEFINITION 
            dsbt=dsb.groups()[0]
            start=True

        if start and f.find(pattern_start)>-1:
            startlist=True

        if startlist:
            fields=fields+f
        if re.search(pattern_end,f):
            start=False
            startlist=False
            if fields:
                srl=sarahlist_to_python(fields.split('=')[-1],only_extract)
                dsbd.update( {dsbt:srl} )
                
                fields=''
    return dsbd

def parse_mathematica_list_of_list(fdotm,NAME='DEFINITION',KEY='EWSB'):
    '''
    Parse mathematica list with the structure:
    
      NAME[KEY][KEYS]={
                             LIST
                             };
    
    and generate a dictionary with KEYS
    '''
    pattern='%s\[%s\]\[(\w+)\]' %(NAME,KEY)
    pattern_start='{'
    pattern_end='\}\s*;'
    only_extract=False
    dsbd=extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract)
    return dsbd

def bidiagonal(w,k='MatterSector'):
    weyl={}
    weyl['left_intr'  ]=w[0][0] #list
    weyl['right_intr' ]=w[0][1] #list
    weyl['left_mass' ]=w[1][0][0] 
    weyl['left_rota'  ]=w[1][0][1]
    weyl['right_mass']=w[1][1][0]
    weyl['right_rota' ]=w[1][1][1]
    if k=='MatterSector':
        weyl['lorentz' ]='WeylFermion'
        #If Diagonal then Scalar or Majorana
    return weyl

def diagonal(s,k='GaugeSector'):
    symm={}
    symm['intr']=s[0]
    symm['mass']=s[1]
    symm['rota']=s[2]
    if k=='GaugeSector':
        symm['Lorentz']='Vector'
    return symm    

def get_vev(v,k='VEVs'):        
    Vev={}
    Vev['Complex']=v[0]
    Vev['vev']=v[1][0]
    Vev['Imaginary']=v[2][0]
    Vev['Real']=v[3][0]
    Vev['vev_coeff']=v[1][1]
    Vev['Imaginary_coeff']=v[2][1]
    Vev['Real_coeff']=v[3][1]
    Vev['Lorentz']='Scalar'
    return Vev

### New defintions

In [71]:
def get_hypercharge(field,particles):
    try: 
        Y=particles[particles['Field']==field].reset_index(
              ).loc[0].get('Properties').get('Groups'
                                   )[0]
    except KeyError:
        Y=None
    return Y

def get_Lorentz(field,particles):
    try: 
        Y=particles[particles['Field']==field].reset_index(
              ).loc[0].get('Properties').get('Lorentz'
                                   )
    except KeyError:
        Y=None
    return Y

def get_higgs_vev(smdict,k,particles):
    '''
    Get the vev associated to the Yukawa coupling
    in `smdict` with Description `k`
    '''
    try:
        H=smdict.get(k).get('Higgs')
    except AttributeError:
        H=''
    if H:
        H0=get_H0(H,particles)
        hh=get_hh(H0,particles)
        if not hh.empty:
            v=hh.get('Properties').apply(lambda d: d.get('vev')).loc[0]
    else:
        v=''
    return v

def get_diagonal_basis(vev,p,particles):
    try:
        pp=particles[particles['Parents']==p].reset_index(drop=True).loc[0,'Field']
        DF=particles[particles['Parents']==pp].reset_index(drop=True).loc[0,'Field']
    except:
        DF=''
    if vev and DF:
        db=r'''Sqrt[2]/%s* {{Mass[%s,1],0,0 },
                {0, Mass[%s,2],0},
                {0, 0, Mass[%s,3]}}''' %(vev,DF,DF,DF)
    else:
        db=''
    return db

def get_multiplet(WF,particles):
    '''WF: Weyl Fermion'''
    mltp=particles[particles['Parents']==WF]
    if mltp.shape[0]==2:
        chiral='Left'
    elif mltp.shape[0]==1:
        chiral='Right'
    else:
        chiral=None
    j=0
    multiplet={}
    for p in mltp['Field']:
        multiplet[p]={}
        j=j+1
        multiplet[p]['chiral']=chiral
        if chiral=='Left':
            multiplet[p]['dim']='doublet'
            if j==1:
                multiplet[p]['pos']='Up'
            elif j==2:
                multiplet[p]['pos']='Down'
            else:
                multiplet[p]['pos']=None
        else:
            multiplet[p]['dim']='singlet'
    return multiplet

def sorted_equality(l1,l2):
    return sorted(l1)==sorted(l2)

def get_H0(H,particles):
    "H is string"
    Hs=particles[particles['Parents']==H]
    #Get doublet components
    if Hs.shape[0]==2:
        # extract neutral component
        for Hf in Hs.get('Field'):
            H0s=particles[particles.get('Parents')==Hf].reset_index(drop=True)
            if not H0s.get('Properties').empty:
                return H0s
        
def get_hh(H0,particles):
    "H0 is a dataframe"
    if not H0.empty:
        return H0[H0.get('Properties').apply(lambda d: d.get('CP')=='Real')].reset_index(drop=True)
    else:
        return pd.DataFrame()

## 2) Get parameters from `DEFINITION[GaugeES]`

In [72]:
dsbd=parse_mathematica_list_of_list(smf,NAME='DEFINITION',KEY='GaugeES')

fdotm=smf
d={}
for lag in dsbd['LagrangianInput']:
    pattern='^\s*\t*({}.*)'.format(lag[0])
    pattern_start='='
    pattern_end=';'
    only_extract=True
    l=extract_code_block(fdotm,pattern,pattern_start,pattern_end,only_extract)
    fl=list(l.values())[0]

    fll=re.sub('[\+\-]','::',
        re.sub('\)*\s*\t*\;\s*\t*$','',
            re.sub( '^\s*\t*[\-\+\(]+','',
            fl))).split('::')

    for x in fll:
        flli=re.sub('^\s*\t*', '',x)
        dd=re.sub('([\w\\\/\s\[\]]+)\s+([\w\[\]\.]+)',r'\1::\2',  flli).split('::')
        d[dd[0].strip()]=dd[1].strip()

In [73]:
d

{'1/2 \\[Lambda]': 'conj[H].H.conj[H].H',
 'Yd': 'conj[H].d.q',
 'Ye': 'conj[H].e.l',
 'Yu': 'u.q.H',
 'mu2': 'conj[H].H'}

In [74]:
particles=pd.read_json('parseparticles.json')

In [75]:
dd={}
for k in d.keys():
    prtcls=[ s.strip() for s in re.sub( '[\w]+\[(\w+)\]',r'\1',d[k]).split('.')]
    print(prtcls)
    dd[k]={'operator':d[k],'fields':prtcls,
           'hypercharge': sorted( [ get_hypercharge(f,particles).replace('-','') 
                                 for f in prtcls]),
           'Lorentz':[ get_Lorentz(f,particles) for f in prtcls]
          }
    
for k in dd.keys():
    ck=re.sub('^[0-9\/\s]+',r'',k)
    if ck!=k:
        vk=dd.pop(k)
        dd[ck]=vk    

['u', 'q', 'H']
['H', 'H']
['H', 'H', 'H', 'H']
['H', 'e', 'l']
['H', 'd', 'q']


In [76]:
dd

{'Yd': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'operator': 'conj[H].d.q'},
 'Ye': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'operator': 'conj[H].e.l'},
 'Yu': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'fields': ['u', 'q', 'H'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'operator': 'u.q.H'},
 '\\[Lambda]': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'operator': 'conj[H].H.conj[H].H'},
 'mu2': {'Lorentz': ['Scalar', 'Scalar'],
  'fields': ['H', 'H'],
  'hypercharge': ['1/2', '1/2'],
  'operator': 'conj[H].H'}}

## Extract the SM copupling from Model file
Check the Yukawa interactions to extract the SM Yukawa couplings, and identify the Higgs field to get the scalar couplings

In [77]:
smdict={'Down-Yukawa-Coupling':{'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
                                'hypercharge': ['1/2', '1/3', '1/6'],
                                'update_Description':{}},
        'Up-Yukawa-Coupling':{'Lorentz': ['WeylFermion', 'WeylFermion','Scalar'],
                              'hypercharge': ['1/2', '1/6', '2/3'],
                             'update_Description':{}},
        'Lepton-Yukawa-Coupling':{'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
                                  'hypercharge': ['1', '1/2', '1/2'],
                                 'update_Description':{}},
        'SM Mu Parameter':  {'Lorentz': ['Scalar', 'Scalar'],
                             'hypercharge': ['1/2', '1/2'],
                             'update_Description':{'OutputName':'m2SM'}},
  'SM Higgs Selfcouplings': {'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
                             'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
                            'update_Description':{}}
       }


In [78]:
smdict

{'Down-Yukawa-Coupling': {'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {}},
 'Lepton-Yukawa-Coupling': {'Lorentz': ['Scalar',
   'WeylFermion',
   'WeylFermion'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {}},
 'SM Higgs Selfcouplings': {'Lorentz': ['Scalar',
   'Scalar',
   'Scalar',
   'Scalar'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Lorentz': ['Scalar', 'Scalar'],
  'hypercharge': ['1/2', '1/2'],
  'update_Description': {'OutputName': 'm2SM'}},
 'Up-Yukawa-Coupling': {'Lorentz': ['WeylFermion', 'WeylFermion', 'Scalar'],
  'hypercharge': ['1/2', '1/6', '2/3'],
  'update_Description': {}}}

In [79]:
# Get standard model Higgs
import sys
for k in dd.keys():
    if (sorted_equality( smdict['Up-Yukawa-Coupling']['Lorentz'],dd[k]['Lorentz'] ) and
        sorted_equality( smdict['Up-Yukawa-Coupling']['hypercharge'],dd[k]['hypercharge'] ) ):
        #Get Higgs from scalar part
        for i in range(len(dd[k]['Lorentz'])):
            if dd[k]['Lorentz'][i]=='Scalar':
                H=dd[k]['fields'][i]
                smdict['Up-Yukawa-Coupling']['Coupling']=k
                smdict['Up-Yukawa-Coupling']['Higgs']=H
                smdict['Up-Yukawa-Coupling']['fields']=dd[k]['fields']
        #Use obtained Higgs to obtain diagonal form
        for i in range(len(dd[k]['Lorentz'])):                
            if dd[k]['Lorentz'][i]=='WeylFermion':
                mltp=get_multiplet(dd[k]['fields'][i],particles)
                for p in mltp.keys():
                    if mltp[p].get('pos')=='Up':
                        vev=get_higgs_vev(smdict,'Up-Yukawa-Coupling',particles)
                        smdict['Up-Yukawa-Coupling'
                              ]['update_Description'
                              ]['DependenceNum']=get_diagonal_basis(
                                                 vev,p,particles)                
                
if not smdict.get('Up-Yukawa-Coupling'):
    sys.exit('"Up-Yukawa-Coupling" NOT FOUND!' )
lk=list(dd.keys())
lk.remove( smdict['Up-Yukawa-Coupling']['Coupling'] )

lds=list(smdict.keys())
lds.remove('Up-Yukawa-Coupling')
for ds in lds:
    for k in lk:
        if (sorted_equality( smdict[ds]['Lorentz'],dd[k]['Lorentz'] ) and
            sorted_equality( smdict[ds]['hypercharge'],dd[k]['hypercharge'] ) ):
            smdict[ds]['Coupling']=k
            smdict[ds]['fields']=dd[k]['fields']
            if H:
                smdict[ds]['Higgs']=H
            for i in range(len(dd[k]['Lorentz'])):
                if dd[k]['Lorentz'][i]=='WeylFermion':
                    mltp=get_multiplet(dd[k]['fields'][i],particles)
                    for p in mltp.keys():
                        if mltp[p].get('pos')=='Down':
                            print(p,mltp[p].get('pos'))
                            vev=get_higgs_vev(smdict,ds,particles)
                            smdict[ds]['update_Description'
                                  ]['DependenceNum']=get_diagonal_basis(
                                                     vev,p,particles)                    

dL Down
eL Down


In [80]:
smdict

{'Down-Yukawa-Coupling': {'Coupling': 'Yd',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
 'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
 'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Coupling': 'mu2',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar'],
  'f

In [81]:
rotation={}
for k in smdict.keys():
    print(k)
    for i in range(len(smdict[k].get('fields'))):
        if smdict[k]['Lorentz'][i]=='WeylFermion':
            mltp=particles[particles['Parents']==smdict[k]['fields'][i]]
            if mltp.shape[0]==2:
                chiral='Left'
            elif mltp.shape[0]==1:
                chiral='Right'
            else:
                chiral=None
                
            j=0
            for p in mltp['Field']:
                j=j+1
                print(p)
                #Updage smdict
                #rotations
                prt=particles.dropna(subset=['Parents'])
                rot=prt[prt['Parents'].dropna(
                       ).str.contains(p)].reset_index()['rotation'].loc[0]
                if type(rot)==str:
                    rotation[p]={rot:{}}
                    dscr='Mixing-Matrix'
                    if chiral=='Left':
                        if j==1:
                            dscr='{}-Up-{}'.format(chiral,dscr)
                        elif j==2:
                            dscr='{}-Down-{}'.format(chiral,dscr)
                    elif chiral=='Right':
                        if re.search('^[uU]',p):
                            dscr='{}-Up-{}'.format(chiral,dscr)
                        elif re.search('^[dD]',p):
                            dscr='{}-Down-{}'.format(chiral,dscr)
                        elif re.search('^[eE]',p):
                            dscr='{}-Lepton-{}'.format(chiral,dscr)
                        else:
                            dscr=None
                    rotation[p][rot]['Description']=dscr

Down-Yukawa-Coupling
dR
uL
dL
Lepton-Yukawa-Coupling
eR
vL
eL
SM Mu Parameter
Up-Yukawa-Coupling
uR
uL
dL
SM Higgs Selfcouplings


In [82]:
smdict

{'Down-Yukawa-Coupling': {'Coupling': 'Yd',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'd', 'q'],
  'hypercharge': ['1/2', '1/3', '1/6'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
 'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
  'fields': ['H', 'e', 'l'],
  'hypercharge': ['1', '1/2', '1/2'],
  'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
 'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
  'fields': ['H', 'H', 'H', 'H'],
  'hypercharge': ['1/2', '1/2', '1/2', '1/2'],
  'update_Description': {}},
 'SM Mu Parameter': {'Coupling': 'mu2',
  'Higgs': 'H',
  'Lorentz': ['Scalar', 'Scalar'],
  'f

In [83]:
rotation

{'dL': {'Vd': {'Description': 'Left-Down-Mixing-Matrix'}},
 'dR': {'Ud': {'Description': 'Right-Down-Mixing-Matrix'}},
 'eL': {'Ve': {'Description': 'Left-Down-Mixing-Matrix'}},
 'eR': {'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'}},
 'uL': {'Vu': {'Description': 'Left-Up-Mixing-Matrix'}},
 'uR': {'Uu': {'Description': 'Right-Up-Mixing-Matrix'}}}

In [84]:
particles[particles['Block']=='GaugeSector']['Properties']

18                {'Lorentz': 'Vector', 'Mass_basis': ['VP', 'VZ'], 'Interaction_basis': ['VB', 'VWB[3]']}
19                {'Lorentz': 'Vector', 'Mass_basis': ['VP', 'VZ'], 'Interaction_basis': ['VB', 'VWB[3]']}
20    {'Lorentz': 'Vector', 'Mass_basis': ['VWp', 'conj[VWp]'], 'Interaction_basis': ['VWB[1]', 'VWB[2]']}
Name: Properties, dtype: object

In [85]:
grps=particles[particles['Definition']=='GaugeES']['Properties'].apply(lambda d: d.get('Group')).dropna()

G={}
coupling={}
for g in grps:
    G=particles[particles['Properties'].apply(lambda d: d.get('Group')==g)].reset_index(drop=True)
    V=G.reset_index().loc[0,'Field']
    print('*',V)
    try:
        VV=particles[particles['Parents'].astype(str).str.contains(V)
                    ].reset_index(drop=True)
        if VV.shape[0]==2:
            VV=VV[VV.get('Properties').astype(str).str.contains(
                   'conj\[\w+\]')].reset_index(drop=True)
    
        f=VV.loc[0,'Field']
        r=VV.loc[0,'rotation']
    except KeyError:
        f=''    
    c=G.loc[0,'Properties']['Coupling'].strip()
    if g.find('U[1]')>-1:
        if f:
            rotation[f]={r: {'Description':"Photon-Z Mixing Matrix"}}
        coupling[c]={'Description':'Hypercharge-Coupling'}
    if g.find('SU[2]')>-1:
        print(g)
        if f:
            rotation[f]={r: {'Description':"W Mixing Matrix"}}
        coupling[c]={'Description':'Left-Coupling'}
    if g.find('SU[3]')>-1:
        coupling[c]={'Description':'Strong-Coupling'}

    #break
    print(g)

* VB
   U[1]
* VWB
 SU[2]
 SU[2]
* VG
  SU[3]


In [86]:
rotation

{'VP': {'ZZ': {'Description': 'Photon-Z Mixing Matrix'}},
 'VWp': {'ZW': {'Description': 'W Mixing Matrix'}},
 'dL': {'Vd': {'Description': 'Left-Down-Mixing-Matrix'}},
 'dR': {'Ud': {'Description': 'Right-Down-Mixing-Matrix'}},
 'eL': {'Ve': {'Description': 'Left-Down-Mixing-Matrix'}},
 'eR': {'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'}},
 'uL': {'Vu': {'Description': 'Left-Up-Mixing-Matrix'}},
 'uR': {'Uu': {'Description': 'Right-Up-Mixing-Matrix'}}}

{\[Lambda],  { Description -> "SM Higgs Selfcouplings",
               DependenceNum -> Mass[hh]^2/(v^2)}},
{v,          { Description -> "EW-VEV",
               DependenceNum -> Sqrt[4*Mass[VWp]^2/(g2^2)],
               DependenceSPheno -> None,
               OutputName -> vvSM}},
{mH2,        { Description -> "SM Higgs Mass Parameter"}},

{ThetaW,    { Description -> "Weinberg-Angle",
              DependenceNum -> ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]  }},

In [87]:
H=smdict['SM Higgs Selfcouplings'].get('Higgs')
if H:
    H0=get_H0(H,particles)
    if not H0.empty:
        hh=get_hh(H0,particles).get('Field').loc[0]
        vev=get_hh(H0,particles).get('Properties').loc[0].get('vev')
        if hh and vev:
            smdict['SM Higgs Selfcouplings'
                  ]['update_Description']={
                    'DependenceNum': r'Mass[{}]^2/({}^2)'.format(hh,vev) }

In [88]:
smdict['EW-VEV']={}
smdict['EW-VEV']['Coupling']=vev
smdict['EW-VEV']['update_Description']={}
smdict['EW-VEV']['update_Description']['DependenceSPheno']='None'
smdict['EW-VEV']['update_Description']['OutputName']='vvSM'
VWp=''
g2=''
for k in rotation.keys():
    if list( rotation[k].values() )[0].get('Description')=='W Mixing Matrix':
        VWp=k
for c in coupling.keys():
    if coupling[c].get('Description')=='Left-Coupling':
        g2=c
smdict['EW-VEV']['update_Description']['DependenceNum']=r'Sqrt[4*Mass[{}]^2/({}^2)]'.format(VWp,g2)

In [89]:
for k in rotation:
    if list( rotation[k].values() )[0].get('Description')=='Photon-Z Mixing Matrix':
        VP=k
        ZZ=list(rotation[k].keys())[0]
fz=particles[particles['rotation']==ZZ]
VZ=fz[fz.get('Field')!=VP].reset_index(drop=True).loc[0,'Field']
smdict['Weinberg-Angle']={}
smdict['Weinberg-Angle']['Coupling']='ThetaW'
smdict['Weinberg-Angle']['update_Description']={}
smdict['Weinberg-Angle']['update_Description'
     ]['DependenceNum']='ArcSin[Sqrt[1 - Mass[{}]^2/Mass[{}]^2]]'.format(
                                                           VWp,VZ )

In [90]:
constants={}
constants['AlphaS']= { 'Description'  : 'Alpha Strong'}
constants['e']     = { 'Description'  :  'electric charge'} 
constants['Gf']    = { 'Description'  :  "Fermi's constant"}
constants['aEWinv']= { 'Description'  :  'inverse weak coupling constant at mZ'}
constants['mH2']   = { 'Description'  :  'SM Higgs Mass Parameter'}

In [91]:
smdict,coupling,rotation,constants

({'Down-Yukawa-Coupling': {'Coupling': 'Yd',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'd', 'q'],
   'hypercharge': ['1/2', '1/3', '1/6'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}'}},
  'EW-VEV': {'Coupling': 'v',
   'update_Description': {'DependenceNum': 'Sqrt[4*Mass[VWp]^2/(g2^2)]',
    'DependenceSPheno': 'None',
    'OutputName': 'vvSM'}},
  'Lepton-Yukawa-Coupling': {'Coupling': 'Ye',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'WeylFermion', 'WeylFermion'],
   'fields': ['H', 'e', 'l'],
   'hypercharge': ['1', '1/2', '1/2'],
   'update_Description': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}'}},
  'SM Higgs Selfcouplings': {'Coupling': '\\[Lambda]',
   'Higgs': 'H',
   'Lorentz': ['Scalar', 'Scalar', 'Scalar', 'Scalar'],
   'fields': ['H', 'H', 'H'

## Build `parameters` with the same standards as `particles`

In [92]:
ParameterDefinitions={'Properties':{}}
ParameterDefinitions['Properties'].update(coupling)
ParameterDefinitions['Properties'].update(constants)
for k in rotation.keys():
    ParameterDefinitions['Properties'].update(rotation.get(k))

#ParameterDefinition

In [93]:
 list(smdict.keys())

['Weinberg-Angle',
 'Down-Yukawa-Coupling',
 'Lepton-Yukawa-Coupling',
 'Up-Yukawa-Coupling',
 'SM Higgs Selfcouplings',
 'EW-VEV',
 'SM Mu Parameter']

In [94]:
vev

'v'

In [96]:
#k='Down-Yukawa-Coupling'
for k in smdict.keys():
    smp={}
    smp['Description']=k
    smp.update(smdict[k].get('update_Description'))
    ParameterDefinitions['Properties'].update( { smdict[k].get('Coupling'):  smp} )
    print("*"*30)
    print( { smdict[k].get('Coupling'):  smp} )
    print("*"*30)
    #break

******************************
{'ThetaW': {'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]', 'Description': 'Weinberg-Angle'}}
******************************
******************************
{'Yd': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}', 'Description': 'Down-Yukawa-Coupling'}}
******************************
******************************
{'Ye': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}', 'Description': 'Lepton-Yukawa-Coupling'}}
******************************
******************************
{'Yu': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 },\n                {0, Mass[Fu,2],0},\n                {0, 0, Mass[Fu,3]}}', 'Description': 'Up-Yukawa-Coupling'}}
******************************
******************************
{'\\[Lambda]': {'DependenceNum': 'Mass[hh]^2/(v^2)', 'Description': 'SM Higgs Selfcouplings'}}
***

In [97]:
ParameterDefinitions

{'Properties': {'AlphaS': {'Description': 'Alpha Strong'},
  'Gf': {'Description': "Fermi's constant"},
  'ThetaW': {'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]',
   'Description': 'Weinberg-Angle'},
  'Ud': {'Description': 'Right-Down-Mixing-Matrix'},
  'Ue': {'Description': 'Right-Lepton-Mixing-Matrix'},
  'Uu': {'Description': 'Right-Up-Mixing-Matrix'},
  'Vd': {'Description': 'Left-Down-Mixing-Matrix'},
  'Ve': {'Description': 'Left-Down-Mixing-Matrix'},
  'Vu': {'Description': 'Left-Up-Mixing-Matrix'},
  'Yd': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 },\n                {0, Mass[Fd,2],0},\n                {0, 0, Mass[Fd,3]}}',
   'Description': 'Down-Yukawa-Coupling'},
  'Ye': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fe,1],0,0 },\n                {0, Mass[Fe,2],0},\n                {0, 0, Mass[Fe,3]}}',
   'Description': 'Lepton-Yukawa-Coupling'},
  'Yu': {'DependenceNum': 'Sqrt[2]/v* {{Mass[Fu,1],0,0 },\n                {0, Mass[Fu,2],0},\n                {0, 0, 

In [98]:
pd.DataFrame(ParameterDefinitions)

,Properties
AlphaS,{'Description': 'Alpha Strong'}
Gf,{'Description': 'Fermi's constant'}
ThetaW,"{'DependenceNum': 'ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]', 'Description': 'Weinberg-Angle'}"
Ud,{'Description': 'Right-Down-Mixing-Matrix'}
Ue,{'Description': 'Right-Lepton-Mixing-Matrix'}
Uu,{'Description': 'Right-Up-Mixing-Matrix'}
Vd,{'Description': 'Left-Down-Mixing-Matrix'}
Ve,{'Description': 'Left-Down-Mixing-Matrix'}
Vu,{'Description': 'Left-Up-Mixing-Matrix'}
Yd,"{'DependenceNum': 'Sqrt[2]/v* {{Mass[Fd,1],0,0 }, {0, Mass[Fd,2],0}, {0, 0, Mass[Fd,3]}}', 'Description': 'Down-Yukawa-Coupling'}"


## Write file

In [99]:
f=open('SMp.json','w')
json.dump(ParameterDefinitions,f)
f.close()

In [100]:
CHECK_PANDAS_SAVED_JSON=True
if CHECK_PANDAS_SAVED_JSON:
    SM=pd.read_json('SMp.json')#,lines=True,orient='records')

In [101]:
SM.columns

Index(['Properties'], dtype='object')

In [102]:
SM.columns
c='Properties'
SM[c].dropna().index

Index(['AlphaS', 'Gf', 'ThetaW', 'Ud', 'Ue', 'Uu', 'Vd', 'Ve', 'Vu', 'Yd',
       'Ye', 'Yu', 'ZW', 'ZZ', '\[Lambda]', 'aEWinv', 'e', 'g1', 'g2', 'g3',
       'mH2', 'mu2', 'v'],
      dtype='object')

In [120]:
d

{'1/2 \\[Lambda]': 'conj[H].H.conj[H].H',
 'Yd': 'conj[H].d.q',
 'Ye': 'conj[H].e.l',
 'Yu': 'u.q.H',
 'mu2': 'conj[H].H'}

In [129]:
from collections import OrderedDict 

d=SM.loc['ThetaW','Properties']
element='Description'
def order_dict_by(d,element='Description'):
    l=list(d.keys())
    if element in l:
        l.remove(element)
        l=[element]+l
        od=OrderedDict()
        for k in l:
            od[k]=d[k]
    return od

In [131]:
order_dict_by(d,element='Description').keys()

odict_keys(['Description', 'DependenceNum'])

In [119]:
['Description']+l

['Description', 'DependenceNum']

In [111]:
from collections import OrderedDict 
od = OrderedDict() 
od['a'] = 1
od['b'] = 2
od['c'] = 3
od['d'] = 4

In [132]:
import json
definitions='ParameterDefintions'#ParticleDefinitions'
f=open('para.m','w')
for c in SM.columns:
    if definitions=='ParticleDefinitions':
        f.write('{}[{}] = {{\n'.format(definitions,c))
    else:
        f.write('{} = {{\n'.format(definitions))
        
    cindex=SM[c].dropna().index
    csep=','
    for p in cindex:
        SMcp=order_dict_by(SM[c][p],element='Description')
        if p==list(cindex)[-1]:
            csep=''
        f.write('    {{{}, {{'.format(p))
        cpkeys=SMcp.keys()
        sangria=' '
        sep=','
        nl='\n'
        #TODO: Be sure that Description will be printed first
        for k in cpkeys:
            if k!=list(cpkeys)[0]:
                sangria='           '
            if k==list(cpkeys)[-1]:
                sep=''
                nl=''
            #print list of strings with double quotes
            cp='{}{} -> '.format(sangria,k)
            smcpk=SM[c][p][k]
            if type(smcpk)==str:
                smcpk='"{}"'.format(smcpk)
            else:
                smcpk=json.dumps( smcpk )
            
            f.write('{}{}{}{}'.format(cp,smcpk,sep,nl))

        f.write('}}}}{}\n'.format(csep))
    f.write('};\n\n')
f.close()

In [133]:
cat para.m

ParameterDefintions = {
    {AlphaS, { Description -> "Alpha Strong"}},
    {Gf, { Description -> "Fermi's constant"}},
    {ThetaW, { Description -> "Weinberg-Angle",
           DependenceNum -> "ArcSin[Sqrt[1 - Mass[VWp]^2/Mass[VZ]^2]]"}},
    {Ud, { Description -> "Right-Down-Mixing-Matrix"}},
    {Ue, { Description -> "Right-Lepton-Mixing-Matrix"}},
    {Uu, { Description -> "Right-Up-Mixing-Matrix"}},
    {Vd, { Description -> "Left-Down-Mixing-Matrix"}},
    {Ve, { Description -> "Left-Down-Mixing-Matrix"}},
    {Vu, { Description -> "Left-Up-Mixing-Matrix"}},
    {Yd, { Description -> "Down-Yukawa-Coupling",
           DependenceNum -> "Sqrt[2]/v* {{Mass[Fd,1],0,0 },
                {0, Mass[Fd,2],0},
                {0, 0, Mass[Fd,3]}}"}},
    {Ye, { Description -> "Lepton-Yukawa-Coupling",
           DependenceNum -> "Sqrt[2]/v* {{Mass[Fe,1],0,0 },
                {0, Mass[Fe,2],0},
                {0, 0, Mass[Fe,3]}}"}},
    {Yu, { Description -> "Up-Yukawa-Coupling",
       

In [142]:
def sarahlist_to_python(strl,DEBUG=False):
    '''
    Convert a string with a SARAH list of rotations into
    a Python object
    '''
    #General transformations
    nl=re.sub(';\s*$','', #Drop final semicolon
               strl)
    if re.search('^\{.*\{.*\{.*->True',strl ) or re.search('^\{.*\{.*\{.*->False',strl ):
        nl=re.sub( '([\w]+)([,:])',r'"\1"\2', # Keep True and False
              re.sub( '[\s\t]+','', # strip
              re.sub( '\s*->\s*',':', # to python dict                  
                  nl))).replace('}},{','}],['
                      ).replace('{{','[['
                      ).replace('}}}','}]]'
                      )        
    elif nl.find('->')>-1:
        nl=re.sub( '(\w+\[*\w+\]*)',r'"\1"' ,
           re.sub('\s*:\s*\{([\s\w,\[\]]+)\}',r':[\1 ]', # to python value lists of dict key
           re.sub( '\s*->\s*',':', # to python dict                  
              nl)))
    else:
        nl= re.sub( '\{','[',
            re.sub( '\}',']',                   
            re.sub('([\w\[\]\/\\\]+)',r'"\1"',  #to python_lists
           nl)))
    if DEBUG:    
        print(nl)
        print("*"*50)
    return eval(nl)

In [141]:
DEFINITION={}
DEFINITION['EWSB']={}
Bidiagonal=[]
Diagonal=[]
VEV=[]
for k in dsbd.keys():
    
    for w in dsbd[k]:
        if k=='GaugeSector' or k=='MatterSector':
            if np.array(w).shape==(2,2):
                weyl=bidiagonal(w,k)
                Bidiagonal.append(weyl)
            elif np.array(w).shape==(3,):
                symm=diagonal(w,k)
                Diagonal.append(symm)
        elif k=='VEVs':
            VEV.append(get_vev(w,k))

DEFINITION['EWSB']['Bidiagonal']=Bidiagonal
DEFINITION['EWSB']['Diagonal']=Diagonal
DEFINITION['EWSB']['VEVs']=VEV
DEFINITION['EWSB']['DiracSpinors']=dsbd['DiracSpinors']

KeyError: 'DiracSpinors'

In [14]:
particles=particles+rotations_to_particles(rotations=DEFINITION,key='EWSB',lr='',sep='')
particles=particles+rotations_to_particles(rotations=DEFINITION,key='EWSB',lr='left',sep='_')
particles=particles+rotations_to_particles(rotations=DEFINITION,key='EWSB',lr='right',sep='_')

In [18]:
newparticles=[]
for d in DEFINITION['EWSB']['VEVs']:
    particle=vev_to_particles(d,cp='Real')
    newparticles.append(particle)
    particle=vev_to_particles(d,cp='Imaginary')
    newparticles.append(particle)
particles=particles+newparticles    

In [20]:
newparticles=[]
dict_of_spinors=lf=DEFINITION['EWSB']['DiracSpinors']
for f in dict_of_spinors.keys():
    particle=spinor_to_particles(dict_of_spinors,f)
    newparticles.append(particle)
particles=particles+newparticles    

In [21]:
pd.DataFrame( particles )

,Block,DEFINITION,Definition,Defintion,Field,Parents,Properties,rotation
0,NaN,NaN,GaugeES,NaN,VB,None,"{'Index': ' hypercharge', 'Group': ' U[1]', ...",NaN
1,NaN,NaN,GaugeES,NaN,VWB,None,"{'Index': ' left', 'Group': ' SU[2]', 'Lorentz...",NaN
2,NaN,NaN,GaugeES,NaN,VG,None,"{'Index': ' color', 'Group': ' SU[3]', 'Loren...",NaN
3,NaN,NaN,WeylFermionAndIndermediate,NaN,q,None,"{'Groups': [' 1/6', ' 2', ' 3; '], 'Lore...",NaN
4,NaN,NaN,WeylFermionAndIndermediate,NaN,l,None,"{'Groups': [' -1/2', ' 2', ' 1;'], 'Lorent...",NaN
5,NaN,NaN,WeylFermionAndIndermediate,NaN,d,None,"{'Groups': [' 1/3', ' 1', ' -3'], 'Lorentz...",NaN
6,NaN,NaN,WeylFermionAndIndermediate,NaN,u,None,"{'Groups': [' -2/3', ' 1', ' -3'], 'Lorentz...",NaN
7,NaN,NaN,WeylFermionAndIndermediate,NaN,e,None,"{'Groups': [' 1', ' 1', ' 1'], 'Lorentz...",NaN
8,NaN,NaN,WeylFermionAndIndermediate,NaN,H,None,"{'Groups': [' 1/2', ' 2', ' 1;'], 'Lorent...",NaN
9,NaN,NaN,WeylFermionAndIndermediate,NaN,uL,q,"{'NF': '3', 'Lorentz': 'WeylFermion'}",NaN


In [13]:
print(smf)

Off[General::spell]

Model`Name = "SM";
Model`NameLaTeX ="Standard Model";
Model`Authors = "F.Staub";
Model`Date = "2014-11-06";

(* 2013-01-24: changed normalization of lambda term to convention of hep-ph/0207271 *)
(* 2013-06-24: using new name conventions (without inital "S" and "F" for scalar and matter fields) *)
(* 2013-09-01: changing to new conventions for FermionFields/MatterFields *)
(* 2014-11-06: Changed sign in Lagrangian to fit standard conventions *)
(* 2016-05-03: Changed sign of Yu *)



(*-------------------------------------------*)
(*   Particle Content*)
(*-------------------------------------------*)

(* Gauge Groups *)

Gauge[[1]]={B,   U[1], hypercharge, g1,False};
Gauge[[2]]={WB, SU[2], left,        g2,True};
Gauge[[3]]={G,  SU[3], color,       g3,False};


(* Matter Fields *)

FermionFields[[1]] = {q, 3, {uL, dL},     1/6, 2,  3};  
FermionFields[[2]] = {l, 3, {vL, eL},    -1/2, 2,  1};
FermionFields[[3]] = {d, 3, conj[dR],     1/3, 1, -3};
FermionFields[[4]] 

In [236]:
1+1

2